In [1]:
import numpy as np
import torch
import EnchoderTRF
from torch.utils.data import TensorDataset, DataLoader
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
data = np.load('Data.npz')

In [3]:
x_train, y_train = shuffle(data['x_train'], data['y_train'])
x_test, y_test = shuffle(data['x_test'], data['y_test'])

In [4]:
x_train

array([[[8.26575947e+00, 2.99046722e+02, 2.80796528e+01, 9.75370312e+00],
        [7.68309212e+00, 2.34550720e+02, 2.56173916e+01, 8.73284054e+00],
        [8.26316833e+00, 2.30724045e+02, 3.13004799e+01, 9.54106998e+00],
        ...,
        [2.44945736e+01, 2.88655548e+02, 0.00000000e+00, 7.61781645e+00],
        [2.08043003e+01, 2.11710220e+02, 4.29655647e+01, 7.68041563e+00],
        [1.69532681e+01, 2.69684174e+02, 1.10479584e+01, 7.42162991e+00]],

       [[8.06956577e+00, 1.89597427e+02, 0.00000000e+00, 9.49316788e+00],
        [1.43989363e+01, 1.50741638e+02, 1.58094854e+01, 1.20000000e+01],
        [1.76057510e+01, 2.26727341e+02, 0.00000000e+00, 7.79253435e+00],
        ...,
        [1.51365910e+01, 2.68538910e+02, 4.81167324e-02, 7.23719454e+00],
        [1.39321194e+01, 2.70556274e+02, 9.74259853e+00, 1.10207529e+01],
        [7.40474272e+00, 2.60619232e+02, 1.66871662e+01, 8.53296852e+00]],

       [[2.61791840e+01, 8.50952072e+01, 0.00000000e+00, 1.20000000e+01],
        

In [5]:
x_scaler = StandardScaler()
shape = x_train.shape
x_train = x_scaler.fit_transform(x_train.reshape(-1, shape[-1]))
x_train = x_train.reshape(shape)
shape = x_test.shape
x_test = x_scaler.transform(x_test.reshape(-1, shape[-1]))
x_test = x_test.reshape(shape)

In [6]:
y_scaler = StandardScaler()
shape = y_train.shape
y_train = y_scaler.fit_transform(y_train.reshape(-1, shape[-1]))
y_train = y_train.reshape(shape)
shape = y_test.shape
y_test = y_scaler.transform(y_test.reshape(-1, shape[-1]))
y_test = y_test.reshape(shape)

In [7]:
def cvt_data_loader(x,y=None,batch_size= 32,shuffle = True):
    if y is None:
        return DataLoader(torch.tensor(x,dtype = torch.float32),batch_size= batch_size,shuffle=shuffle)
    
    return DataLoader(TensorDataset(torch.tensor(x,dtype = torch.float32),
                              torch.tensor(y,dtype = torch.float32))
                              ,batch_size= batch_size,shuffle=shuffle)

In [8]:
train_dl = cvt_data_loader(x_train, y_train, 128, True)
test_dl = cvt_data_loader(x_test, y_test, 128, True)

In [9]:
model = EnchoderTRF.EncoderOnly(4)

In [10]:
critarion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.002)

In [11]:
torch.cuda.is_available()

True

In [12]:
device = torch.device('cuda')

In [13]:
epochs = 200
model = model.to(device)
model = model.train()
train_loss = []
test_loss = []
min_loss = 10000;
for epoch in range(1, epochs + 1):
    print(f"Epoch : {epoch}")
    sum_loss = 0
    model = model.train()
    for x, y in tqdm.tqdm(train_dl):
        optimizer.zero_grad()
        x = x.to(device)
        y = y.to(device)
        out = model.forward(x)
        loss = critarion(y, out)
        loss.backward()
        torch.nn.utils.clip_grad_value_(model.parameters(),torch.tensor(1))
        optimizer.step()
        sum_loss += float(loss) * len(x)
    print(f"Train Loss : {sum_loss/ len(train_dl.dataset)}")
    train_loss.append(sum_loss / len(train_dl.dataset))
    sum_loss = 0
    model = model.eval()
    for x, y in tqdm.tqdm(test_dl):
        optimizer.zero_grad()
        x = x.to(device)
        y = y.to(device)
        out = model.forward(x)
        loss = critarion(y, out)
        sum_loss += float(loss) * len(x)
        if sum_loss < min_loss:
            min_loss = sum_loss
            torch.save(model.state_dict(),"PredTRF")
    print(f"Val Loss : {sum_loss/ len(test_dl.dataset)}")
    test_loss.append(sum_loss / len(test_dl.dataset))

Epoch : 1


100%|██████████| 192/192 [00:01<00:00, 134.13it/s]


Train Loss : 0.8401599017228711


100%|██████████| 90/90 [00:00<00:00, 256.63it/s]


Val Loss : 0.7753372883380241
Epoch : 2


...

Val Loss : 0.8345952054090375
Epoch : 200


100%|██████████| 192/192 [00:01<00:00, 154.61it/s]


Train Loss : 0.6776086729995587


100%|██████████| 90/90 [00:00<00:00, 402.40it/s]

Val Loss : 0.8435430427617902


In [14]:
train_loss

[0.8401599017228711,
 0.7854970410496542,
 0.7813865730524548,
 0.77793481243118,
 0.7767338660923867,
 0.7756540137735743,
 0.775685859862753,
 0.7747884053465305,
 0.7744407466084311,
 0.7740671536053271,
 0.773957876892051,
 0.7739056528901374,
 0.7737378102358878,
 0.7731536872566596,
 0.7730246896947717,
 0.7730553912630878,
 0.7726153603882022,
 0.7723927885160426,
 0.772008239572257,
 0.7719504785246373,
 0.7714767384626229,
 0.7711811578055263,
 0.7711423187974514,
 0.7704253609068767,
 0.770559427752757,
 0.7704483691543765,
 0.7699483925504743,
 0.7698677912603328,
 0.7689449753198021,
 0.7689601093445679,
 0.7685944464298955,
 0.7679918919852697,
 0.7674539623872326,
 0.7669813110872101,
 0.7669574649270828,
 0.7663601665098653,
 0.7657922617432542,
 0.765521943792549,
 0.7647381124525595,
 0.7642591555006878,
 0.7637659649596438,
 0.762686634641558,
 0.7621921903538365,
 0.7612832930714438,
 0.7606916836128701,
 0.7604671514932589,
 0.7591428366649418,
 0.7586801730056887,


In [15]:
test_loss

[0.7753372883380241,
 0.772133028663402,
 0.7696845654525091,
 0.7675044162200528,
 0.7660457067718672,
 0.7688394908405287,
 0.7686712573709447,
 0.7657458163557094,
 0.7652220300920145,
 0.7665847540214072,
 0.7673010324078355,
 0.7676239742462292,
 0.7675310112503418,
 0.7665371867454729,
 0.7686737009635659,
 0.7682201765614306,
 0.7671550841206547,
 0.7677170112768115,
 0.7683624750974397,
 0.7699561897740094,
 0.767242271650306,
 0.770540839626279,
 0.768397370290548,
 0.7696134502294282,
 0.7687373265428835,
 0.767434971259671,
 0.7707966705180672,
 0.7701174045025522,
 0.7694035044611802,
 0.7699029500203362,
 0.771186253826691,
 0.7703428311118913,
 0.7711587117436671,
 0.7709805352510843,
 0.7701674072815341,
 0.771629047008581,
 0.7722980135913499,
 0.7716148743254649,
 0.7741621561550157,
 0.7728193034251184,
 0.7736388181807172,
 0.775580283897933,
 0.7751544300229268,
 0.7727747685524053,
 0.7737736217423818,
 0.7731748936165889,
 0.7771980551965372,
 0.7791497877903901,


In [17]:
dic = {
    "train_loss":np.array(train_loss),
    'val_loss':np.array(test_loss)
}

In [19]:
np.savez("LossTRF",**dic)

In [20]:
np.load('LossTRF.npz')

NpzFile 'LossTRF.npz' with keys: train_loss, val_loss